In [ ]:
# use pyodbc; install ODBC Driver 17,18 for SQL Server for ubuntu 22.04
!lsb_release -a
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo tee /etc/apt/trusted.gpg.d/microsoft.asc
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
!sudo apt-get install -y unixodbc-dev
!pip install pyodbc

import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from google.cloud import storage

# 日期參數
DateStart = '20240801'
DateEnd = '20240805'

In [ ]:
# 替換成你的 SQL Server 連接參數
server = '125.227.50.167,6666'
database = 'master'
username = 'cmda'
password = '1234'

# # prompt: 連線SQl Server
# conn = pyodbc.connect('Driver={ODBC Driver 18 for SQL Server};'
#                       f'Server={server};'
#                       f'Database={database};'
#                       f'UID={username};'
#                       f'PWD={password};'
#                       'TrustServerCertificate=yes;'
#                       )

# # 測試連線
# cursor = conn.cursor()
# df = pd.read_sql_query('SELECT @@VERSION', conn)
# print(df)
# conn.close()

# 使用 SQLAlchemy 創建資料庫引擎
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

# 測試連線
query = 'SELECT @@VERSION'
df = pd.read_sql(query, engine)
print(df)

In [13]:
# 產生交易日期列表
str_query = f"""
SELECT 日期 AS OpenDate
 FROM [CMSERVER].[northwind].[dbo].[sysdbase]
WHERE 股票代號 = 'TWA00'
 AND 日期 between {DateStart} and {DateEnd}
ORDER BY 日期
"""
df_date = pd.read_sql(str_query, engine)

In [48]:
# 查詢語法
str_query = """
DECLARE @Ddate as varchar(8)
DECLARE @expire as tinyint
DECLARE @expireDate as varchar(8)
SET @Ddate = '{openDate}'        --手動參數
SET @expire = (
        SELECT  MIN([離到期日(天)])
          FROM  [CMSERVER].[northwind].[dbo].[sysoption]
         WHERE  代號 LIKE 'TXO[0-9][0-9][0-9][0-9][0-9][0-9][C-P]%'
           AND  日期 = @Ddate
        )
SET @expireDate = (
        SELECT  MIN(到期日)
          FROM  [CMSERVER].[northwind].[dbo].[sysoption]
         WHERE  代號 LIKE 'TXO[0-9][0-9][0-9][0-9][0-9][0-9][C-P]%'
           AND  日期 = @Ddate
           AND  [離到期日(天)] = @expire
        );

--日期對照表
WITH DateMapping AS
(
SELECT  *
        ,CASE WHEN r = 0 THEN 'D-Open'
              WHEN 日期 = @expireDate THEN 'D-Close'
              ELSE 'D-' + cast(r as varchar(2))
              END AS Hold_Days
  FROM (
        SELECT  日期
                ,ROW_NUMBER() OVER(ORDER BY 日期) - 1 AS r
          FROM  [CMSERVER].[northwind].[dbo].[sysdbase]
         WHERE  股票代號 = 'TWA00'
           AND  日期 BETWEEN @Ddate and @expireDate
        ) t
 WHERE  r IN (0,1,2,3,4,5,10,15,20)
    OR  日期 = @expireDate
)
--商品表
,symbol AS
(
SELECT  代號, Pickout
  FROM (
        SELECT  A.*
                ,CASE WHEN A.結算價 = B.結算價 THEN '價外1檔'
                      WHEN ROW_NUMBER() OVER(PARTITION BY A.買賣權 ORDER BY ABS(A.結算價 - 300)) = 1 THEN '300'
                      WHEN ROW_NUMBER() OVER(PARTITION BY A.買賣權 ORDER BY ABS(A.結算價 - 200)) = 1 THEN '200'
                      WHEN ROW_NUMBER() OVER(PARTITION BY A.買賣權 ORDER BY ABS(A.結算價 - 100)) = 1 THEN '100'
                      WHEN ROW_NUMBER() OVER(PARTITION BY A.買賣權 ORDER BY ABS(A.結算價 - 50)) = 1 THEN '50'
                      ELSE null
                      END AS Pickout
          FROM (
                SELECT  代號, 買賣權, 履約價, 結算價
                  FROM  [CMSERVER].[northwind].[dbo].[sysoption]
                 WHERE  代號 LIKE 'TXO[0-9][0-9][0-9][0-9][0-9][0-9][C-P]%'
                   AND  日期 = @Ddate
                   AND  [離到期日(天)] = @expire
                ) A
         INNER  JOIN (
                SELECT  買賣權, 履約價, 結算價
                  FROM  [CMSERVER].[northwind].[dbo].[sysoption]
                 WHERE  代號 LIKE 'TXO[0-9][0-9][0-9][0-9][0-9][0-9][C-P]%'
                   AND  日期 = @Ddate
                   AND  [離到期日(天)] = @expire
                   AND  檔位 = '價外1檔'
                ) B
            ON (A.買賣權 = B.買賣權 AND A.結算價 <= B.結算價)
        ) t
 WHERE  Pickout is not null
)
--日淨值表變數
,DailyPNL AS
(
SELECT  開倉日, 買賣權, 履約價, Pickout, [D-Open], [D-1], [D-2], [D-3], [D-4], [D-5], [D-10], [D-15], [D-20], [D-Close]
  FROM (
        SELECT  @Ddate AS 開倉日, A.買賣權, A.履約價, B.Pickout, C.Hold_Days, A.結算價
          FROM (
                SELECT  *
                  FROM  [CMSERVER].[northwind].[dbo].[sysoption]
                 WHERE  代號 LIKE 'TXO[0-9][0-9][0-9][0-9][0-9][0-9][C-P]%'
                   AND  日期 BETWEEN @Ddate and (SELECT MAX(日期) FROM DateMapping)
                ) A
         INNER  JOIN symbol B
            ON  A.代號 = B.代號
         INNER  JOIN DateMapping C
            ON  A.日期 = C.日期
        ) S
 PIVOT (SUM(結算價) FOR Hold_Days IN ([D-Open], [D-1], [D-2], [D-3], [D-4], [D-5], [D-10], [D-15], [D-20], [D-Close])) AS P
)

--統計
SELECT  CONVERT(varchar(10), cast(開倉日 as date), 121) AS OpenDate
        ,(
            SELECT  [收盤價]
              FROM  [CMSERVER].[northwind].[dbo].[sysdbase]
             WHERE  股票代號 = 'TWA00'
               AND  日期 = @Ddate
            ) AS TAIEX
		,(
            SELECT  [漲幅(%)] / 100.0
              FROM  [CMSERVER].[northwind].[dbo].[sysdbase]
             WHERE  股票代號 = 'TWA00'
               AND  日期 = @Ddate
            ) AS ROI_TAIEX
		,(
            SELECT  [收盤價]
              FROM  [CMSERVER].[northwind].[dbo].[sysOptFlucClose]	--台指選擇權波動率
             WHERE  代號 = 'VIX15'
               AND  日期 = @Ddate
            ) AS VIX
        ,Pickout
        ,SUM([D-Open]) AS [D-Open]
        ,SUM([D-1]) AS [D-1]
        ,SUM([D-2]) AS [D-2]
        ,SUM([D-3]) AS [D-3]
        ,SUM([D-4]) AS [D-4]
        ,SUM([D-5]) AS [D-5]
        ,SUM([D-10]) AS [D-10]
        ,SUM([D-15]) AS [D-15]
        ,SUM([D-20]) AS [D-20]
        ,SUM([D-Close]) AS [D-Close]
        ,SUM([D-1]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-1]
        ,SUM([D-2]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-2]
        ,SUM([D-3]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-3]
        ,SUM([D-4]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-4]
        ,SUM([D-5]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-5]
        ,SUM([D-10]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-10]
        ,SUM([D-15]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-15]
        ,SUM([D-20]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-20]
        ,SUM([D-Close]) * 1.0 / SUM([D-Open]) - 1 AS [ROI_D-Close]
  FROM  DailyPNL
 GROUP  BY 開倉日, Pickout
"""

In [49]:
# 迴圈查詢
for OpenDate in df_date['OpenDate']:
  df = pd.read_sql(str_query.format(openDate=OpenDate), engine)

  # prompt: 判斷 df_result 是否存在
  if 'df_result' in locals():
    df_result = pd.concat([df_result, df], ignore_index=True)
  else:
    df_result = df.copy()

In [ ]:
# 將 DataFrame 轉換為 Parquet 格式
df_result.to_parquet('Option_ROI_in_Large_Volatility.parquet', index=False)

# Initialize a client
storage_client = storage.Client()

# 上傳到 Cloud Storage
bucket_name = 'meng_workspace'
file_path = 'Option_ROI_in_Large_Volatility.parquet'
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob('sample/Option_ROI_in_Large_Volatility.parquet')
blob.upload_from_filename(file_path)

print(f'File {file_path} uploaded to bucket {bucket_name}')